In [28]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
style.use('fivethirtyeight')

#preprocessing
import nltk
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for part-of-speech tagging
from nltk import pos_tag


In [2]:
import DataCleaning_reviews_5

In [3]:
df5 = DataCleaning_reviews_5.data_clean('../reviews_Grocery_and_Gourmet_Food_5.json.gz')

151254
{'reviewerID': 'A1VEELTKS8NLZB', 'asin': '616719923X', 'reviewerName': 'Amazon Customer', 'helpful': [0, 0], 'reviewText': 'Just another flavor of Kit Kat but the taste is unique and a bit different.  The only thing that is bothersome is the price.  I thought it was a bit expensive....', 'overall': 4.0, 'summary': 'Good Taste', 'unixReviewTime': 1370044800, 'reviewTime': '06 1, 2013'}
151254


In [4]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151254 entries, 0 to 151253
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   reviewerID     151254 non-null  object 
 1   asin           151254 non-null  object 
 2   reviewerName   149761 non-null  object 
 3   reviewText     151254 non-null  object 
 4   helpful_votes  151254 non-null  object 
 5   rating         151254 non-null  float64
 6   reviewTitle    151254 non-null  object 
dtypes: float64(1), object(6)
memory usage: 9.2+ MB


In [5]:
df = df5[['reviewText','reviewTitle', 'asin']]

In [6]:
df.head()

,reviewText,reviewTitle,asin
0,Just another flavor of Kit Kat but the taste i...,Good Taste,616719923X
1,I bought this on impulse and it comes from Jap...,"3.5 stars, sadly not as wonderful as I had hoped",616719923X
2,Really good. Great gift for any fan of green t...,Yum!,616719923X
3,"I had never had it before, was curious to see ...",Unexpected flavor meld,616719923X
4,I've been looking forward to trying these afte...,"Not a very strong tea flavor, but still yummy ...",616719923X


In [7]:
# check for null values
print(df['reviewText'].isnull().sum())
df['reviewTitle'].isnull().sum()  # no null values.

print(df.shape)
df.head()

0
(151254, 3)


,reviewText,reviewTitle,asin
0,Just another flavor of Kit Kat but the taste i...,Good Taste,616719923X
1,I bought this on impulse and it comes from Jap...,"3.5 stars, sadly not as wonderful as I had hoped",616719923X
2,Really good. Great gift for any fan of green t...,Yum!,616719923X
3,"I had never had it before, was curious to see ...",Unexpected flavor meld,616719923X
4,I've been looking forward to trying these afte...,"Not a very strong tea flavor, but still yummy ...",616719923X


In [8]:
df["all_text"] = df["reviewText"] + ' ' + df["reviewTitle"]


<ipython-input-8-fab1430bca7a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["all_text"] = df["reviewText"] + ' ' + df["reviewTitle"]


In [9]:
df.head()

,reviewText,reviewTitle,asin,all_text
0,Just another flavor of Kit Kat but the taste i...,Good Taste,616719923X,Just another flavor of Kit Kat but the taste i...
1,I bought this on impulse and it comes from Jap...,"3.5 stars, sadly not as wonderful as I had hoped",616719923X,I bought this on impulse and it comes from Jap...
2,Really good. Great gift for any fan of green t...,Yum!,616719923X,Really good. Great gift for any fan of green t...
3,"I had never had it before, was curious to see ...",Unexpected flavor meld,616719923X,"I had never had it before, was curious to see ..."
4,I've been looking forward to trying these afte...,"Not a very strong tea flavor, but still yummy ...",616719923X,I've been looking forward to trying these afte...


In [24]:
stemmer_porter=PorterStemmer()
from nltk.stem.snowball import SnowballStemmer
import string  
snowball_stem = SnowballStemmer(language='english')
s_words = set(stopwords.words('english'))
punc = set(string.punctuation)
def is_only_alpha(text):
    return " ".join([word for word in text.split() if word.isalpha()])
def stemmers(llst):
    return [stemmer_porter.stem(words) for words in llst]
df['cleanText1']= df['all_text'].apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
df['cleanText2'] = df['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))
df['cleanText3'] = df['cleanText2'].apply(is_only_alpha)
df['cleanText4'] = df['cleanText3'].apply(word_tokenize)
df['cleanText5'] = df['cleanText4'].apply(lambda words: [word.lower() for word in words])
df['cleanText6'] = df['cleanText5'].apply(stemmers)
df['cleanText7'] = df['cleanText6'].apply(lambda words: " ".join(words))
df = df.drop(['cleanText1','cleanText2','cleanText3','cleanText4','cleanText5','cleanText6'],axis=1)

['i', 'make', 'claims', 'sports', 'nutrition', 'or', 'nutrition', 'matter', 'expertise', 'i', 'll', 'call', 'snack', 'leave', 'place', 'exercise', 'program', 'personal', 'doctor', 'nutritionist', 'coach', 'guru', 'however', 'i', 'say', 'much', 'better', 'average', 'tasting', 'sports', 'bar', 'snack', 'name', 'preference', 'free', 'cardboad', 'ish', 'stale', 'texture', 'barely', 'identifiable', 'aftertastes', 'common', 'similar', 'products', 'while', 'noone', 'compare', 'real', 'apple', 'crisp', 'convenient', 'food', 'go', 'recognizable', 'tastes', 'oats', 'this', 'flavor', 'texture', 'predominates', 'apple', 'cinnamon', 'due', 'large', 'part', 'i', 'suspect', 'fact', 'actually', 'contains', 'ingredients', 'there', 'also', 'raisins', 'cane', 'sugars', 'including', 'fructose', 'fruit', 'juices', 'giving', 'sweet', 'cloying', 'as', 'say', 'wineries', 'taste', 'this', 'company', 'known', 'quality', 'ingredients', 'generally', 'consumer', 'friendly', 'face', 'read', 'powerbar', 'com', 'the'

In [ ]:
vectorizer = TfidfVectorizer(max_features=10_000, lowercase=False)
X = vectorizer.fit_transform(df['cleanText7'])

In [ ]:
X[10:,10:].todense()

In [186]:
# X is the tf-idf of the reviews 
# y is the target or the inputed review from the user
user_input = "I want a marinara sauce that tastes similar to how my mom makes it with fresh tomatoes, garlic, basil, and olive oil."
user_df = pd.DataFrame({'user1': [user_input]})
def is_only_alpha(text):
    return " ".join([word for word in text.split() if word.isalpha()])
def stemmers(llst):
    return [stemmer_porter.stem(words) for words in llst]
user_df['cleanText1']= user_df['user1'].apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
user_df['cleanText2'] = user_df['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))
user_df['cleanText3'] = user_df['cleanText2'].apply(is_only_alpha)
user_df['cleanText4'] = user_df['cleanText3'].apply(word_tokenize)
user_df['cleanText5'] = user_df['cleanText4'].apply(lambda words: [word.lower() for word in words])
user_df['cleanText6'] = user_df['cleanText5'].apply(stemmers)
user_df['cleanText7'] = user_df['cleanText6'].apply(lambda words: " ".join(words))
#print(user_df['cleanText7'])
#user_df = df.drop(['cleanText1','cleanText2','cleanText3','cleanText4','cleanText5','cleanText6'],axis=1)
y = vectorizer.transform([user_df['cleanText7'][0]])

0    i want marinara sauc tast similar mom make fre...
Name: cleanText7, dtype: object


In [153]:
from sklearn.metrics.pairwise import cosine_similarity
sim_matrix = cosine_similarity(y,X)
sim_sorted= np.argsort(sim_matrix)[::-1] # idx of lowest value to highest value but with ::-1 its decending
sim_sorted

array([[75626, 87463, 36845, ..., 38110,  2959, 15071]])

In [154]:
df['cleanText7'][sim_sorted[0][-2]]

'these realli great tomato i hate metal tast can tomato tast like chanc leach tomato can t healthi these rich flavor wateri great sauc virgin oliv oil fresh basil garlic s p cook pasta firm finish cook sauc simpl delici love tomato wish i could find local least i get amazon ever without amazon no tinni metal tast'

array([[103184,  62671,  33582, ...,  74295, 117112,  70191]])

'i love to cook so i bought it for when price get to high for me to buy it ani more love it'

In [176]:
item_asin_top1 = df.iloc[sim_sorted[0][-8]].loc['asin']
item_asin_top1
item_asin_top10 = []
for i in range(10):
    item_asin_top10.append(df.iloc[sim_sorted[0][-i]].loc['asin'])
top10_url = []
for asin in item_asin_top10:
    url = "http://www.amazon.com/dp/"+asin
    top10_url.append(url)
top10_url

# now I have to find if one of these is glutine free, 
#organic or keto and stuff like that. also try to find if I can 
# find broken links by cleaining up the reviews with only ones in meta
# berfore that I have to use meta to find the ones that have organic or glutine free
# then clean up reviews using only the ones that i have in meta after the fact
# and then clean up the recomendations so it doesnt rocomend only the same item like 
# pop tars and tomato sause

['http://www.amazon.com/dp/B002TMV3E4',
 'http://www.amazon.com/dp/B000EZ7J9G',
 'http://www.amazon.com/dp/B0005ZVOR8',
 'http://www.amazon.com/dp/B0012HP1CC',
 'http://www.amazon.com/dp/B000LKZ9IC',
 'http://www.amazon.com/dp/B000EZ7J9G',
 'http://www.amazon.com/dp/B000LKXG64',
 'http://www.amazon.com/dp/B001SAYKJS',
 'http://www.amazon.com/dp/B000F3Q4AM',
 'http://www.amazon.com/dp/B000MMO8W2']

In [117]:
import DataCleaning_meta

In [118]:
df_meta = DataCleaning_meta.data_clean_meta("../meta_Grocery_and_Gourmet_Food.json.gz")

287051
{'category': ['Grocery & Gourmet Food', 'Dairy, Cheese & Eggs', 'Cheese', 'Gouda'], 'tech1': '', 'description': ['BEEMSTER GOUDA CHEESE AGED 18/24 MONTHS', 'Statements regarding dietary supplements have not been evaluated by the FDA and are not intended to diagnose, treat, cure, or prevent any disease or health condition.'], 'fit': '', 'title': 'Beemster Gouda - Aged 18/24 Months - App. 1.5 Lbs', 'also_buy': [], 'image': [], 'tech2': '', 'brand': 'Ariola Imports', 'feature': [], 'rank': '165,181 in Grocery & Gourmet Food (', 'also_view': ['B0000D9MYM', 'B0000D9MYL', 'B00ADHIGBA', 'B00H9OX598', 'B001LM42GY', 'B001LM5TDY'], 'main_cat': 'Grocery', 'similar_item': '', 'date': '', 'price': '$41.91', 'asin': '0681727810'}
287051


In [172]:
df_meta[df_meta['asin'] == item_asin_top1]

,description,brand,rank,price,asin,name,features,categories,main_category
12553,[Fresh. Fast. Easy. Gluten free. Vegetarian. W...,Scarpetta,[],$28.40,B000F3Q4AM,"Scarpetta Marinara Sauce, 19.8-Ounce Jars (Pac...",[],"[Grocery & Gourmet Food, Sauces, Gravies & Mar...",Grocery


In [177]:
df.head()

,reviewText,reviewTitle,asin,all_text,cleanText6,cleanText7
0,Just another flavor of Kit Kat but the taste i...,Good Taste,616719923X,Just another flavor of Kit Kat but the taste i...,"[just, anoth, flavor, kit, kat, tast, uniqu, b...",just anoth flavor kit kat tast uniqu bit diffe...
1,I bought this on impulse and it comes from Jap...,"3.5 stars, sadly not as wonderful as I had hoped",616719923X,I bought this on impulse and it comes from Jap...,"[i, bought, impuls, come, japan, amus, famili,...",i bought impuls come japan amus famili weird s...
2,Really good. Great gift for any fan of green t...,Yum!,616719923X,Really good. Great gift for any fan of green t...,"[realli, good, great, gift, fan, green, tea, j...",realli good great gift fan green tea just expe...
3,"I had never had it before, was curious to see ...",Unexpected flavor meld,616719923X,"I had never had it before, was curious to see ...","[i, never, befor, curiou, see, like, smooth, g...",i never befor curiou see like smooth great sub...
4,I've been looking forward to trying these afte...,"Not a very strong tea flavor, but still yummy ...",616719923X,I've been looking forward to trying these afte...,"[i, ve, look, forward, tri, hear, popular, jap...",i ve look forward tri hear popular japan among...


In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 151254 entries, 0 to 151253
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   reviewText   151254 non-null  object
 1   reviewTitle  151254 non-null  object
 2   asin         151254 non-null  object
 3   all_text     151254 non-null  object
 4   cleanText6   151254 non-null  object
 5   cleanText7   151254 non-null  object
dtypes: object(6)
memory usage: 13.1+ MB


In [197]:
df_meta['description'][0]

['BEEMSTER GOUDA CHEESE AGED 18/24 MONTHS',
 'Statements regarding dietary supplements have not been evaluated by the FDA and are not intended to diagnose, treat, cure, or prevent any disease or health condition.']

In [182]:
df_meta_life = df_meta[df_meta['main_category'] == 'Grocery']

In [198]:
df_meta_life['all_text'] = df_meta_life['description'].astype(str) + ' ' + df_meta_life['name'].astype(str) + " " + df_meta_life['categories'].astype(str)

<ipython-input-198-419902078dea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_life['all_text'] = df_meta_life['description'].astype(str) + ' ' + df_meta_life['name'].astype(str) + " " + df_meta_life['categories'].astype(str)


In [204]:
df_meta_life.drop(['clean_all_text'],axis = 1, inplace=True)

/home/nico/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [205]:
def is_only_alpha(text):
    return " ".join([word for word in text.split() if word.isalpha()])
def stemmers(llst):
    return [stemmer_porter.stem(words) for words in llst]
df_meta_life['cleanText1'] = df_meta_life['description'].apply(lambda words: " ".join(words)).apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
df_meta_life['cleanText2'] = df_meta_life['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))
df_meta_life['cleanText3'] = df_meta_life['cleanText2'].apply(is_only_alpha)
df_meta_life['cleanText4'] = df_meta_life['cleanText3'].apply(word_tokenize)
df_meta_life['cleanText5'] = df_meta_life['cleanText4'].apply(lambda words: [word.lower() for word in words])
df_meta_life['cleanText5'][0]

<ipython-input-205-f2e0972a3538>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_life['cleanText1'] = df_meta_life['description'].apply(lambda words: " ".join(words)).apply(lambda words:" ".join([word for word in words.split() if word not in s_words]))
<ipython-input-205-f2e0972a3538>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_meta_life['cleanText2'] = df_meta_life['cleanText1'].apply(lambda words:" ".join(["".join([c if c not in punc else " " for c in word]) for word in words.split()]))
<

KeyboardInterrupt: 

In [195]:
df_meta_life['cleanText5'][0]

['beemster',
 'gouda',
 'cheese',
 'aged',
 'months',
 'statements',
 'regarding',
 'dietary',
 'supplements',
 'evaluated',
 'fda',
 'intended',
 'diagnose',
 'treat',
 'cure',
 'prevent',
 'disease',
 'health',
 'condition']

In [20]:
def lifestyle(df):
    lifestyles = ['vegetarian','paleo Diet','keto', 'vegan','lowsugar','gluten free','low fat',"organic", "nongmo"]
    